# VO2max Prediction Using Treadmill Maximal Exercise Tests and Machine Learning Techniques

In [2]:
# import packages
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import signal
from collections import Counter
from sklearn.utils import indexable
from sklearn.utils import resample
from sklearn.utils.validation import _num_samples
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from itertools import chain
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV, MultiTaskLassoCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from math import ceil, floor

In [38]:
# load subject dataset
subject_data = pd.read_csv('/Users/luisekutzner/AEEB/project_aaeb/physionet.org/files/treadmill-exercise-cardioresp/1.0.1/subject-info.csv')

# basic information
display(subject_data.head())
print("Initial Dataset Info:")
print(subject_data.info())
print(subject_data.describe())

,Age,Weight,Height,Humidity,Temperature,Sex,ID,ID_test
0,10.8,48.8,163.0,39.0,20.7,1,543,543_1
1,11.8,41.0,150.0,41.0,22.3,1,11,11_1
2,12.2,46.0,160.0,37.0,21.5,0,829,829_1
3,13.2,71.0,190.0,49.0,23.8,1,284,284_1
4,13.7,53.8,169.7,40.0,25.3,0,341,341_1


Initial Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 992 entries, 0 to 991
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Age          992 non-null    float64
 1   Weight       992 non-null    float64
 2   Height       992 non-null    float64
 3   Humidity     962 non-null    float64
 4   Temperature  962 non-null    float64
 5   Sex          992 non-null    int64  
 6   ID           992 non-null    int64  
 7   ID_test      992 non-null    object 
dtypes: float64(5), int64(2), object(1)
memory usage: 62.1+ KB
None
              Age      Weight      Height    Humidity  Temperature  \
count  992.000000  992.000000  992.000000  962.000000   962.000000   
mean    28.979133   73.383367  174.913508   48.211435    22.818565   
std     10.076653   12.005361    7.950027    8.560991     2.784066   
min     10.800000   41.000000  150.000000   23.700000    15.000000   
25%     21.100000   66.000000  170.0

In [39]:
# load measurement dataset
measurement_data = pd.read_csv('/Users/luisekutzner/AEEB/project_aaeb/physionet.org/files/treadmill-exercise-cardioresp/1.0.1/test_measure.csv')

# basic information
display(measurement_data.head())
print("Initial Dataset Info:")
print(measurement_data.info())
print(measurement_data.describe())

,time,Speed,HR,VO2,VCO2,RR,VE,ID_test,ID
0,0,5.0,63.0,478.0,360.0,27,13.3,2_1,2
1,2,5.0,75.0,401.0,295.0,23,10.3,2_1,2
2,4,5.0,82.0,449.0,319.0,29,12.2,2_1,2
3,7,5.0,87.0,461.0,340.0,28,12.8,2_1,2
4,9,5.0,92.0,574.0,417.0,28,14.6,2_1,2


Initial Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 575087 entries, 0 to 575086
Data columns (total 9 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   time     575087 non-null  int64  
 1   Speed    575087 non-null  float64
 2   HR       574106 non-null  float64
 3   VO2      570216 non-null  float64
 4   VCO2     570216 non-null  float64
 5   RR       575087 non-null  int64  
 6   VE       575087 non-null  float64
 7   ID_test  575087 non-null  object 
 8   ID       575087 non-null  int64  
dtypes: float64(5), int64(3), object(1)
memory usage: 39.5+ MB
None
                time          Speed             HR            VO2  \
count  575087.000000  575087.000000  574106.000000  570216.000000   
mean      628.126172       9.607958     146.940892    2313.617768   
std       325.588844       4.520384      32.206372     978.103888   
min         0.000000       0.000000       0.000000      -5.000000   
25%       375.000000      

In [40]:
# merge both datasets on ID
merged_data = pd.merge(measurement_data, subject_data, on='ID', how='inner')

display(merged_data.head())
print(merged_data.info())

,time,Speed,HR,VO2,VCO2,RR,VE,ID_test_x,ID,Age,Weight,Height,Humidity,Temperature,Sex,ID_test_y
0,0,5.0,63.0,478.0,360.0,27,13.3,2_1,2,33.8,68.0,171.1,NaN,NaN,0,2_1
1,2,5.0,75.0,401.0,295.0,23,10.3,2_1,2,33.8,68.0,171.1,NaN,NaN,0,2_1
2,4,5.0,82.0,449.0,319.0,29,12.2,2_1,2,33.8,68.0,171.1,NaN,NaN,0,2_1
3,7,5.0,87.0,461.0,340.0,28,12.8,2_1,2,33.8,68.0,171.1,NaN,NaN,0,2_1
4,9,5.0,92.0,574.0,417.0,28,14.6,2_1,2,33.8,68.0,171.1,NaN,NaN,0,2_1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 773618 entries, 0 to 773617
Data columns (total 16 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   time         773618 non-null  int64  
 1   Speed        773618 non-null  float64
 2   HR           772394 non-null  float64
 3   VO2          768747 non-null  float64
 4   VCO2         768747 non-null  float64
 5   RR           773618 non-null  int64  
 6   VE           773618 non-null  float64
 7   ID_test_x    773618 non-null  object 
 8   ID           773618 non-null  int64  
 9   Age          773618 non-null  float64
 10  Weight       773618 non-null  float64
 11  Height       773618 non-null  float64
 12  Humidity     753579 non-null  float64
 13  Temperature  753579 non-null  float64
 14  Sex          773618 non-null  int64  
 15  ID_test_y    773618 non-null  object 
dtypes: float64(10), int64(4), object(2)
memory usage: 94.4+ MB
None


## Feature Selection

The feature selection process is based on the Paper: ... .

### Data Cleaning and Pre-processing

In [41]:
# check if there are missing values
print(merged_data.isnull().sum())

time               0
Speed              0
HR              1224
VO2             4871
VCO2            4871
RR                 0
VE                 0
ID_test_x          0
ID                 0
Age                0
Weight             0
Height             0
Humidity       20039
Temperature    20039
Sex                0
ID_test_y          0
dtype: int64


In [42]:
# remove rows with missing target variable (HR, VO2)
merged_data_cleaned = merged_data.dropna(subset=['HR', 'VO2'])

# Check how many rows are removed
print(f'Rows removed due to missing HR or VO2: {len(merged_data) - len(merged_data_cleaned)}')

Rows removed due to missing HR or VO2: 6083


In [43]:
print("Exercise Data RR Intervals:")
print(merged_data_cleaned['RR'])

Exercise Data RR Intervals:
0         27
1         23
2         29
3         28
4         28
          ..
773613    35
773614    32
773615    29
773616    31
773617    31
Name: RR, Length: 767535, dtype: int64


In [44]:
# remove RR intervals exclusive to the range 300-2000 ms
# those are considered as outliers
# Entferne RR-Werte, die außerhalb des Bereichs von 5-50 Atemzügen pro Minute liegen (Ausreißer)
merged_data_cleaned['RR'] = np.where((merged_data_cleaned['RR'] < 5) | (merged_data_cleaned['RR'] > 50), np.nan, merged_data_cleaned['RR'])

# linear interpolation to fill missing values
merged_data_cleaned['RR'] = merged_data_cleaned['RR'].interpolate(method='linear')

# Check for NaN values after interpolation
print(merged_data_cleaned.isnull().sum())

time               0
Speed              0
HR                 0
VO2                0
VCO2               0
RR                 0
VE                 0
ID_test_x          0
ID                 0
Age                0
Weight             0
Height             0
Humidity       19997
Temperature    19997
Sex                0
ID_test_y          0
dtype: int64


/var/folders/bd/3yf697f17xx34gzbkjr3wz9r0000gn/T/ipykernel_98049/1413795601.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_cleaned['RR'] = np.where((merged_data_cleaned['RR'] < 5) | (merged_data_cleaned['RR'] > 50), np.nan, merged_data_cleaned['RR'])
/var/folders/bd/3yf697f17xx34gzbkjr3wz9r0000gn/T/ipykernel_98049/1413795601.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_cleaned['RR'] = merged_data_cleaned['RR'].interpolate(method='linear')


In [45]:
# remove if HR and VO2max are out of phase
merged_data_cleaned = merged_data_cleaned[(merged_data_cleaned['HR'] > 0) & (merged_data_cleaned['VO2'] > 0)]

# remove where consecutive HR differs more by than 30 bpm_
merged_data_cleaned['HR_diff'] = merged_data_cleaned['HR'].diff().abs()
merged_data_cleaned = merged_data_cleaned[merged_data_cleaned['HR_diff'] <= 30]

# remove objects with less than 5 minutes of data
merged_data_cleaned['time'] = merged_data_cleaned['time'].astype(float)  # make it numeirc
merged_data_cleaned = merged_data_cleaned[merged_data_cleaned['time'] >= 300]

# check dataset
print(f'Dataset size after cleaning: {merged_data_cleaned.shape}')

Dataset size after cleaning: (622931, 17)


In [47]:
# split data into exercise and recovery phases
exercise_data = merged_data_cleaned[merged_data_cleaned['Speed'] >= 5]  # Exercise phase
recovery_data = merged_data_cleaned[merged_data_cleaned['Speed'] < 5]  # Recovery phase

# shapes of dataframes
print(f'Exercise data size: {exercise_data.shape}')
print(f'Recovery data size: {recovery_data.shape}')

Exercise data size: (573861, 17)
Recovery data size: (49070, 17)


In [48]:
# HRV features

# noch von chat gpt 

from scipy.signal import detrend
from typing import Dict, Any

def compute_hrv_features(segment: Dict[str, Any]) -> Dict[str, float]:
    '''
    Berechnet die Herzfrequenzvariabilitäts- (HRV-) Features für einen gegebenen Datenabschnitt.

    Parameter:
    -----------
    segment : Dict[str, Any]
        Ein Dictionary oder DataFrame, das die RR-Intervalldaten enthält. Es wird angenommen, 
        dass die RR-Intervalle unter dem Schlüssel 'RR' als numpy-Array oder pandas-Serie verfügbar sind.

    Rückgabe:
    ---------
    Dict[str, float]
        Ein Dictionary mit den berechneten HRV-Features:
        - RMSSD: Wurzel des mittleren quadratischen Unterschieds zwischen aufeinanderfolgenden RR-Intervallen.
        - PNN20: Prozentsatz der RR-Intervalldifferenzen > 20 ms.
        - PNN50: Prozentsatz der RR-Intervalldifferenzen > 50 ms.
        - NNI_Range: Differenz zwischen maximalem und minimalem RR-Intervall.
        - VLF_Power: Power im Very Low Frequency-Bereich (simuliert).
        - LF_Power: Power im Low Frequency-Bereich (simuliert).
        - HF_Power: Power im High Frequency-Bereich (simuliert).
        - SD1: Kurzfristige Variation (Standardabweichung einer Achse im Poincare-Plot).
        - SD2: Langfristige Variation (Standardabweichung der anderen Achse im Poincare-Plot).
    '''
    # Extrahiere die RR-Intervalle aus dem Segment
    rr_intervals = segment['RR'].values  # Annahme: RR-Intervalle liegen als numpy-Array oder pandas-Serie vor

    # Überprüfe auf NaN/Inf-Werte und bereinige die Daten
    rr_intervals = rr_intervals[np.isfinite(rr_intervals)]
    if len(rr_intervals) == 0:
        raise ValueError("Alle RR-Intervall-Daten wurden aufgrund von NaN/Inf-Werten entfernt.")

    # Zeit-Domain-Features
    rmssd = np.sqrt(np.mean(np.diff(rr_intervals) ** 2))  # Berechnung von RMSSD
    pnn20 = np.mean(np.abs(np.diff(rr_intervals)) > 20) * 100  # Prozentsatz der Differenzen > 20 ms
    pnn50 = np.mean(np.abs(np.diff(rr_intervals)) > 50) * 100  # Prozentsatz der Differenzen > 50 ms
    nni_range = np.max(rr_intervals) - np.min(rr_intervals)  # Bereich zwischen max. und min. RR-Intervall

    # Frequenz-Domain-Features
    total_power = np.var(detrend(rr_intervals))  # Gesamtvarianz nach Detrend
    vlf_power = total_power * 0.1  # Dummy-Verhältnis für VLF-Power (simuliert)
    lf_power = total_power * 0.3  # Dummy-Verhältnis für LF-Power (simuliert)
    hf_power = total_power * 0.6  # Dummy-Verhältnis für HF-Power (simuliert)

    # Nichtlineare Features
    sd1 = np.std(rr_intervals[:len(rr_intervals)//2])  # Kurzfristige Variation (SD1)
    sd2 = np.std(rr_intervals[len(rr_intervals)//2:])  # Langfristige Variation (SD2)

    # Rückgabe der berechneten HRV-Features als Dictionary
    return {
        'RMSSD': rmssd,
        'PNN20': pnn20,
        'PNN50': pnn50,
        'NNI_Range': nni_range,
        'VLF_Power': vlf_power,
        'LF_Power': lf_power,
        'HF_Power': hf_power,
        'SD1': sd1,
        'SD2': sd2
    }

In [49]:
# apply HRV features
exercise_hrv_features = compute_hrv_features(exercise_data)
recovery_hrv_features = compute_hrv_features(recovery_data)

In [50]:
# new dataframe with features

hrv_features = {
    'exercise': exercise_hrv_features,
    'recovery': recovery_hrv_features
}

print("HRV Features (Exercise):", hrv_features['exercise'])
print("HRV Features (Recovery):", hrv_features['recovery'])

HRV Features (Exercise): {'RMSSD': 1.3516204259257965, 'PNN20': 0.03241208657163768, 'PNN50': 0.0, 'NNI_Range': 43.0, 'VLF_Power': 7.786296438106714, 'LF_Power': 23.358889314320137, 'HF_Power': 46.717778628640275, 'SD1': 8.703193567654504, 'SD2': 8.934431679598106}
HRV Features (Recovery): {'RMSSD': 2.247603956532144, 'PNN20': 0.2812366259756669, 'PNN50': 0.0, 'NNI_Range': 44.0, 'VLF_Power': 7.195403998667158, 'LF_Power': 21.58621199600147, 'HF_Power': 43.17242399200294, 'SD1': 8.563326652756075, 'SD2': 8.41849136180928}


## Feature Extraction

In [51]:
from scipy import stats
from scipy.signal import detrend

In [54]:
# auch mit chat GPT lol

# all HRV Features

def compute_hrv_features(segment: pd.DataFrame):
    """Berechnet HRV-Features für einen gegebenen Segment-Datensatz."""
    hr = segment['HR'].values
    time = segment['time'].values
    
    # Berechnung der Zeitdomänen-Merkmale
    nn_20 = np.mean(np.abs(np.diff(hr)) > 20) * 100  # PNN20
    nn_50 = np.mean(np.abs(np.diff(hr)) > 50) * 100  # PNN50
    range_nni = np.max(hr) - np.min(hr)  # Range NNI

    # Berechnung der Frequenzdomänen-Merkmale (VLF, LF, HF)
    total_power = np.var(detrend(hr))  # Gesamtvarianz nach Detrend
    vlf_power = total_power * 0.1  # Dummy-Verhältnis für VLF-Power
    lf_power = total_power * 0.3  # Dummy-Verhältnis für LF-Power
    hf_power = total_power * 0.6  # Dummy-Verhältnis für HF-Power

    # Nichtlineare Merkmale (Poincare Plot, DFA)
    # Für die Poincare-Plot-Merkmale verwenden wir die Standardabweichungen
    sd1 = np.std(hr[:len(hr)//2])
    sd2 = np.std(hr[len(hr)//2:])
    
    # DFA (Detrended Fluctuation Analysis) als Platzhalter (echte Implementierung erfordert spezialisierte Bibliotheken)
    # Dummy-Wert für DFA a1 (z.B. von einer DFA-Analyse)
    dfa_a1 = 0.75  # Beispielwert für DFA a1

    # Berechnung der Steigungen der Merkmale über die Zeit
    # Beispiel: Steigung des HRs über Zeit
    slope_nni = np.polyfit(time, hr, 1)[0]
    
    # Da vlf_power, lf_power und hf_power konstante Werte sind, berechnen wir ihre Steigung nicht
    slope_vlf = np.nan
    slope_lf = np.nan
    slope_hf = np.nan
    
    # Steigungen für Poincare Plot-Merkmale
    slope_sd1 = np.polyfit(time, [sd1] * len(time), 1)[0]
    slope_sd2 = np.polyfit(time, [sd2] * len(time), 1)[0]

    # Zeitbasierte Merkmale (Beispiel)
    max_speed = segment['Speed'].max()
    time_to_25_speed = segment[segment['Speed'] >= 0.25 * max_speed]['time'].iloc[0]
    time_to_25_hr = segment[segment['HR'] >= 0.25 * max(segment['HR'])]['time'].iloc[0]

    # Duration in HR-Bereich 50-60% der maximalen HR
    hr_max = max(segment['HR'])
    hr_min = min(segment['HR'])
    duration_hr60 = segment[(segment['HR'] >= 0.5 * hr_max) & (segment['HR'] <= 0.6 * hr_max)]['time'].sum()

    # Feature Dictionary
    return {
        'NNI_20': nn_20,
        'NNI_50': nn_50,
        'Range_NNI': range_nni,
        'VLF_Power': vlf_power,
        'LF_Power': lf_power,
        'HF_Power': hf_power,
        'SD1': sd1,
        'SD2': sd2,
        'DFA_A1': dfa_a1,
        'Slope_NNI': slope_nni,
        'Slope_VLF': slope_vlf,
        'Slope_LF': slope_lf,
        'Slope_HF': slope_hf,
        'Slope_SD1': slope_sd1,
        'Slope_SD2': slope_sd2,
        'TimeTo25Speed': time_to_25_speed,
        'TimeTo25HR': time_to_25_hr,
        'DurationHR60': duration_hr60
    }



In [55]:
# Beispiel-Daten: Übungs- und Erholungsphasen
exercise_hrv_features = compute_hrv_features(exercise_data)
recovery_hrv_features = compute_hrv_features(recovery_data)

# Konvertieren der extrahierten Merkmale in DataFrame für bessere Übersicht
exercise_features_df = pd.DataFrame([exercise_hrv_features])
recovery_features_df = pd.DataFrame([recovery_hrv_features])

# Anzeigen der extrahierten Merkmale
display(exercise_features_df)
display(recovery_features_df)

,NNI_20,NNI_50,Range_NNI,VLF_Power,LF_Power,HF_Power,SD1,SD2,DFA_A1,Slope_NNI,Slope_VLF,Slope_LF,Slope_HF,Slope_SD1,Slope_SD2,TimeTo25Speed,TimeTo25HR,DurationHR60
0,0.1645,0.048792,202.0,58.154281,174.462843,348.925685,23.834831,24.397219,0.75,0.025824,NaN,NaN,NaN,-6.511974e-15,-7.626134e-15,300.0,300.0,55398809.0


,NNI_20,NNI_50,Range_NNI,VLF_Power,LF_Power,HF_Power,SD1,SD2,DFA_A1,Slope_NNI,Slope_VLF,Slope_LF,Slope_HF,Slope_SD1,Slope_SD2,TimeTo25Speed,TimeTo25HR,DurationHR60
0,0.868165,0.533942,172.0,87.470757,262.412272,524.824545,29.879921,29.676642,0.75,0.03054,NaN,NaN,NaN,-8.196211e-17,-1.232184e-16,300.0,300.0,14524356.0
